In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import xgboost as xgb

# Instantiate XGBClassifier with default settings
xgb_model = xgb.XGBClassifier()

# Check the parameters
params = xgb_model.get_params()
print(params)

{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': None, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


def invert_threshold(image, threshold=0.5):
    inverted_image = tf.where(image > threshold, 0.0, 1.0)  # Invert based on threshold
    return inverted_image


train_dir = '/kaggle/input/gujarati-p-data/gujarati-handwritten-digit-dataset-master/Train Set'
test_dir = '/kaggle/input/gujarati-p-data/gujarati-handwritten-digit-dataset-master/Test Set'


train_datagen = ImageDataGenerator(
    preprocessing_function=lambda x: invert_threshold(x / 255.0)
)

test_datagen = ImageDataGenerator(
    preprocessing_function=lambda x: invert_threshold(x / 255.0)
)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),  
    color_mode='grayscale',  
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(32, 32),  
    color_mode='grayscale',  
    batch_size=32,
    class_mode='categorical'
)


base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(75, 75, 3))


model = models.Sequential([
    layers.Input(shape=(32, 32, 1)),  
    layers.Resizing(75, 75),  
    layers.Conv2D(3, (3, 3), padding='same'),  
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')  
])


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


model.fit(
    train_generator,
    epochs=100,
    validation_data=test_generator,
    callbacks=[early_stopping]
)


test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f'Test Accuracy: {test_acc * 100:.2f}%')


In [2]:
model.save('/kaggle/working/inceptionv3sf.keras')